# Aula 05 - Avaliação de Sistemas de Recomendação - Exercícios

In [1]:
import pandas as pd
import numpy as np

### Importar base de dados

In [2]:
import wget
!python3 -m wget https://github.com/mmanzato/MBABigData/raw/master/ml-20m-compact.tar.gz
!tar -xvzf ml-20m-compact.tar.gz

100% [....................................................] 65019041 / 65019041
Saved under ml-20m-compact.tar.gz
dataset/
dataset/tags_sample.csv
dataset/._.DS_Store
dataset/.DS_Store
dataset/movies_sample.csv
dataset/._genome-tags.csv
dataset/genome-tags.csv
dataset/._ml-youtube.csv
dataset/ml-youtube.csv
dataset/._genome-scores.csv
dataset/genome-scores.csv
dataset/ratings_sample.csv


In [3]:
movies = pd.read_csv('./dataset/movies_sample.csv')
ratings = pd.read_csv('./dataset/ratings_sample.csv')
df = ratings[['userId', 'movieId', 'rating']]
df = df.merge(movies[['movieId', 'title']])
df

,userId,movieId,rating,title
0,11,7481,5.0,Enemy Mine (1985)
1,11,1046,4.5,Beautiful Thing (1996)
2,11,616,4.0,"Aristocats, The (1970)"
3,11,3535,2.0,American Psycho (2000)
4,11,5669,5.0,Bowling for Columbine (2002)
...,...,...,...,...
190616,138493,288,5.0,Natural Born Killers (1994)
190617,138493,1748,5.0,Dark City (1998)
190618,138493,616,4.0,"Aristocats, The (1970)"
190619,138493,1597,4.5,Conspiracy Theory (1997)


In [4]:
movies_genres = movies.drop('genres', axis=1).join(movies.genres.str.split('|', expand=True)
             .stack().reset_index(drop=True, level=1).rename('genre'))
movies_genres.head()

,movieId,title,genre
0,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Crime
0,30,Shanghai Triad (Yao a yao yao dao waipo qiao) ...,Drama
1,31,Dangerous Minds (1995),Drama
2,37,Across the Sea of Time (1995),Documentary
2,37,Across the Sea of Time (1995),IMAX


In [5]:
movies_tags = pd.read_csv('./dataset/tags_sample.csv')
movies_tags.head()

,userId,movieId,tag,timestamp_y
0,279,916,Gregory Peck,1329962459
1,279,916,need to own,1329962471
2,279,916,romantic comedy,1329962476
3,279,916,Rome,1329962490
4,279,916,royalty,1329962474


In [6]:
map_users = {user: idx for idx, user in enumerate(df.userId.unique())}
map_items = {item: idx for idx, item in enumerate(df.movieId.unique())}

df['userId'] = df['userId'].map(map_users)
df['movieId'] = df['movieId'].map(map_items)

movies_tags['userId'] = movies_tags['userId'].map(map_users)
movies_tags['movieId'] = movies_tags['movieId'].map(map_items)
movies_tags.dropna(inplace=True)
movies_tags['movieId'] = movies_tags.movieId.astype(int)
movies_tags['userId'] = movies_tags.userId.astype(int)

movies_genres['movieId'] = movies_genres['movieId'].map(map_items)
movies_genres.dropna(inplace=True)
movies_genres['movieId'] = movies_genres.movieId.astype(int)

map_title = {}
for _, row in df.iterrows():
    map_title[row.movieId] = row.title


In [7]:
movies_genres[['movieId', 'genre']].to_csv('items_genres.dat', index=False, sep='\t', header=False)
movies_tags[['movieId', 'tag']].to_csv('items_tags.dat', index=False, sep='\t', header=False)

### Divisão da base em treino e teste

In [8]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=.2, random_state=2)
train.to_csv('train.dat', index=False, header=False, sep='\t')
test.to_csv('test.dat', index=False, header=False, sep='\t')

### Exemplos de utilização do CaseRecommender

In [10]:
from caserec.recommenders.rating_prediction.item_attribute_knn import ItemAttributeKNN

ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_genres.dat', k_neighbors=10, as_similar_first=True).compute()
ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_tags.dat', k_neighbors=10, as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.716349 sec
>> metadata:: 417 items and 20 metadata (890 interactions) | sparsity:: 89.33%
prediction_time:: 0.531304 sec
Eval:: MAE: 0.73347 RMSE: 0.964352 
[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.732800 sec
>> metadata:: 231 items and 1979 metadata (6274 interactions) | sparsity:: 98.63%
prediction_time:: 0.722296 sec
Eval:: MAE: 0.765116 RMSE: 1.006911 


***Exercício 01:*** Verifique o efeito no RSME ao aumentar o número de vizinhos do algoritmo ItemAttributeKNN com usando gêneros. Explique.

In [11]:
ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_genres.dat', k_neighbors=10, as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.831352 sec
>> metadata:: 417 items and 20 metadata (890 interactions) | sparsity:: 89.33%
prediction_time:: 0.563956 sec
Eval:: MAE: 0.73347 RMSE: 0.964352 


In [16]:
ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_genres.dat', k_neighbors=100, as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.816736 sec
>> metadata:: 417 items and 20 metadata (890 interactions) | sparsity:: 89.33%
prediction_time:: 2.165663 sec
Eval:: MAE: 0.717937 RMSE: 0.945425 


In [17]:
ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_genres.dat', k_neighbors=1000, as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.599906 sec
>> metadata:: 417 items and 20 metadata (890 interactions) | sparsity:: 89.33%
prediction_time:: 3.845947 sec
Eval:: MAE: 0.695346 RMSE: 0.916212 


In [18]:
ItemAttributeKNN('train.dat', 'test.dat', metadata_file='items_genres.dat', k_neighbors=10571O , as_similar_first=True).compute()

[Case Recommender: Rating Prediction > Item Attribute KNN Algorithm]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 4.567529 sec
>> metadata:: 417 items and 20 metadata (890 interactions) | sparsity:: 89.33%
prediction_time:: 3.760998 sec
Eval:: MAE: 0.695346 RMSE: 0.916212 


O RMSE reduz, o que faz sentido para uma recomendação item-based: ao avaliarmos a recomendação de um item i a um usuário u, conforme aumentamos o K, aumentamos os itens que podem ser considerados para fazer o cálculo. Porém, todos os itens considerados (vizinhos) já foram avaliados por u. Então supondo que o usuário é consistente em suas avaliações, considerar mais filmes, ou seja, aumentar o K, pode ser benéfico para reduzir o erro da predição de um usuário, ao custo de um aumento no tempo de predição.



***Exercício 02:*** Verifique o efeito na métrica MAP@1 ao aumentar o número de fatores do algoritmo BPRMF. Explique.

**Resposta**: Olhando os resultados, podemos observar uma queda no MAP@1 conforme aumentamos os fatores, mas essa queda se suaviza a partir de um certo ponto e volta a subir, até estagnar.

É dificil explicar a influência do número de fatores na qualidade da recomendação porque isso é dependente de quão variável são as notas no conjunto de treino/teste, já que um fator representa uma covariância de notas entre usuários. Aumentar o número de fatores pode ajudar a capturar padrões mais sútis nas notas, mas se as notas tiverem um comportamento muito simples de modelar, um número elevado de fatores pode trazer pouquíssimo retorno em qualidade. Ademais, um número muito elevado de fatores pode gerar matrizes de fatores altamente esparsas, o que contraria a principal vantagem de métodos de fatorização de matrizes, que é a redução de dimensionalidade. No exemplo observado, parece que os ganhos estagnaram a partir de quarenta fatores.

In [19]:
from caserec.recommenders.item_recommendation.bprmf import BprMF

In [24]:
BprMF('train.dat', 'test.dat', 'ir_bprmf.dat', factors=1).compute()

[Case Recommender: Item Recommendation > BPRMF]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 131.600548 sec
prediction_time:: 2.330587 sec


Eval:: PREC@1: 0.382651 PREC@3: 0.269133 PREC@5: 0.229326 PREC@10: 0.17479 RECALL@1: 0.123045 RECALL@3: 0.247465 RECALL@5: 0.34519 RECALL@10: 0.5141 MAP@1: 0.382651 MAP@3: 0.46812 MAP@5: 0.475743 MAP@10: 0.451012 NDCG@1: 0.382651 NDCG@3: 0.549101 NDCG@5: 0.57668 NDCG@10: 0.575779 


In [20]:
BprMF('train.dat', 'test.dat', 'ir_bprmf.dat', factors=3).compute()

[Case Recommender: Item Recommendation > BPRMF]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 109.972873 sec
prediction_time:: 2.304375 sec


Eval:: PREC@1: 0.385016 PREC@3: 0.277268 PREC@5: 0.233034 PREC@10: 0.174695 RECALL@1: 0.125738 RECALL@3: 0.256478 RECALL@5: 0.35331 RECALL@10: 0.514036 MAP@1: 0.385016 MAP@3: 0.475578 MAP@5: 0.482271 MAP@10: 0.458216 NDCG@1: 0.385016 NDCG@3: 0.562114 NDCG@5: 0.585653 NDCG@10: 0.58307 


MAP@1 com factors = 3: 0.38

In [21]:
BprMF('train.dat', 'test.dat', 'ir_bprmf.dat', factors=6).compute()

[Case Recommender: Item Recommendation > BPRMF]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 108.232088 sec
prediction_time:: 2.397865 sec


Eval:: PREC@1: 0.368272 PREC@3: 0.269448 PREC@5: 0.230007 PREC@10: 0.176937 RECALL@1: 0.11883 RECALL@3: 0.24959 RECALL@5: 0.347935 RECALL@10: 0.521914 MAP@1: 0.368272 MAP@3: 0.461948 MAP@5: 0.471427 MAP@10: 0.446572 NDCG@1: 0.368272 NDCG@3: 0.550435 NDCG@5: 0.577983 NDCG@10: 0.576309 


MAP@1 com factors = 6: 0.36

In [22]:
BprMF('train.dat', 'test.dat', 'ir_bprmf.dat', factors=9).compute()

[Case Recommender: Item Recommendation > BPRMF]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 110.029202 sec
prediction_time:: 2.440248 sec


Eval:: PREC@1: 0.36988 PREC@3: 0.27834 PREC@5: 0.235474 PREC@10: 0.178678 RECALL@1: 0.120998 RECALL@3: 0.259004 RECALL@5: 0.357146 RECALL@10: 0.528201 MAP@1: 0.36988 MAP@3: 0.466977 MAP@5: 0.476679 MAP@10: 0.45351 NDCG@1: 0.36988 NDCG@3: 0.558307 NDCG@5: 0.585103 NDCG@10: 0.584165 


MAP@1 com factors = 9: 0.36

In [23]:
BprMF('train.dat', 'test.dat', 'ir_bprmf.dat', factors=12).compute()

[Case Recommender: Item Recommendation > BPRMF]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 123.558928 sec
prediction_time:: 2.429651 sec


Eval:: PREC@1: 0.370164 PREC@3: 0.275408 PREC@5: 0.234037 PREC@10: 0.177845 RECALL@1: 0.121659 RECALL@3: 0.255428 RECALL@5: 0.354117 RECALL@10: 0.52563 MAP@1: 0.370164 MAP@3: 0.468278 MAP@5: 0.476966 MAP@10: 0.453784 NDCG@1: 0.370164 NDCG@3: 0.5613 NDCG@5: 0.586181 NDCG@10: 0.583835 


MAP@1 com factors = 12; 0.37

In [25]:
BprMF('train.dat', 'test.dat', 'ir_bprmf.dat', factors=40).compute()

[Case Recommender: Item Recommendation > BPRMF]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 112.878119 sec
prediction_time:: 2.342254 sec


Eval:: PREC@1: 0.41226 PREC@3: 0.300287 PREC@5: 0.251783 PREC@10: 0.184949 RECALL@1: 0.133796 RECALL@3: 0.27982 RECALL@5: 0.382319 RECALL@10: 0.544795 MAP@1: 0.41226 MAP@3: 0.505518 MAP@5: 0.510662 MAP@10: 0.482288 NDCG@1: 0.41226 NDCG@3: 0.594712 NDCG@5: 0.616462 NDCG@10: 0.607938 


In [26]:
BprMF('train.dat', 'test.dat', 'ir_bprmf.dat', factors=100).compute()

[Case Recommender: Item Recommendation > BPRMF]

train data:: 11090 users and 405 items (152496 interactions) | sparsity:: 96.60%
test data:: 10571 users and 331 items (38125 interactions) | sparsity:: 98.91%

training_time:: 112.520154 sec
prediction_time:: 2.316647 sec


Eval:: PREC@1: 0.418314 PREC@3: 0.305143 PREC@5: 0.253126 PREC@10: 0.185195 RECALL@1: 0.136555 RECALL@3: 0.283024 RECALL@5: 0.383296 RECALL@10: 0.544919 MAP@1: 0.418314 MAP@3: 0.510863 MAP@5: 0.514627 MAP@10: 0.486789 NDCG@1: 0.418314 NDCG@3: 0.598977 NDCG@5: 0.618991 NDCG@10: 0.612612 


***Exercício 03:*** Implemente a métrica Average Precision para um único usuário. Utilize as recomendações geradas pelo algoritmo BPRMF (consulte notebook Aula04_Exemplo como base).

In [27]:
recs = pd.read_csv('./ir_bprmf.dat', sep='\t', names=['userId', 'movieId', 'score'])
recs_user = recs.loc[(recs.userId==1), 'movieId'].tolist()
print(recs_user)

[19, 14, 10, 20, 33, 4, 28, 61, 56, 8]


In [29]:
ground_truth = test.loc[(test.userId==1), 'movieId'].tolist()

In [37]:
recs_user[:3]

[19, 14, 10]

In [36]:
ground_truth

[13, 16, 14, 19, 24, 20]

In [48]:
precision_at_k(3, recs_user, ground_truth)

[1, 1, 0]


0.5

In [53]:
def average_precision(user_id, recs_df, test_df):
    recs_user = recs_df.loc[(recs_df.userId==user_id), 'movieId'].tolist()
    ground_truth = test_df.loc[(test_df.userId==user_id), 'movieId'].tolist()

    precisions = 0
    for k in range(1, len(recs_user)):
        precisions += precision_at_k(k, recs_user, ground_truth)
    return precisions/k

In [52]:
def precision_at_k(k, rec_list, ground_truth):
    focus_list = rec_list[:k]
    relevant_list = [1 if item in ground_truth else 0 for item in focus_list]
    n_relevants = 0
    for idx, label in enumerate(relevant_list):
        n_relevants += label / (idx + 1)
    return n_relevants / len(focus_list)

In [54]:
average_precision(1, recs, test)

0.4435956790123457

***Exercício 04:*** Um importante aspecto que pode ser avaliado em Sistemas de Recomendação é a diversidade da lista de recomendações. A métrica Intra-List Similarity (ILS) aplica uma função de similaridade (e.g. Cosseno, Jaccard, Pearson, etc.) entre todos os pares de itens da lista de recomendação, usando seus metadados como gêneros, tags, etc. Mais detalhes sobre essa métrica podem ser encontrados em: https://grouplens.org/site-content/uploads/Improving-WWW-20051.pdf

***a)*** Implemente uma função que calcula a ILS de uma lista de recomendação para um único usuário. Utilize os gêneros de filmes.

In [ ]:
# TODO

***b)*** Utilize a função que implementou no item (a) para calcular a ILS de todos os usuários da base de dados. Utilize as recomendações geradas pelo algoritmo BPR MF do CaseRecommender.

In [ ]:
# TODO

***c)*** Aplique a função que implementou no item (b) para calcular a ILS das recomendações geradas pelos algoritmos ItemKNN e ItemAttributeKNN. Compare os resultados de todos os algoritmos, verificando inclusive o que acontece com a acurácia (precisão, MAP ou NDCG) quando a diversidade aumenta ou diminui.

In [ ]:
# TODO